In [2]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import math

In [8]:
broad_2019 = pd.read_csv("input/broadband_data_2019November.csv")
broad_2020 = pd.read_csv("input/broadband_data_2020October.csv")
broad_challenge = pd.read_csv("input/broadband_data_opendatachallenge.csv")
broad_zip = pd.read_csv("input/broadband_data_zipcode.csv")